In [60]:
import pandas as pd
import math
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

## Загружаем данные

In [2]:
raw_data=pd.read_csv('Data.csv')

In [3]:
raw_data['InvoiceDate'] = pd.to_datetime(raw_data['InvoiceDate'], format='%Y-%m-%d') # Убираем время за ненадобностью
raw_data['InvoiceDate'] = raw_data['InvoiceDate'].dt.strftime('%Y-%m-%d')

In [4]:
raw_data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
944463,575312,22083,PAPER CHAIN KIT RETROSPOT,6,2011-11-09,2.95,13588.0,United Kingdom
944464,575312,23355,HOT WATER BOTTLE KEEP CALM,4,2011-11-09,4.95,13588.0,United Kingdom
944465,575312,22110,BIRD HOUSE HOT WATER BOTTLE,6,2011-11-09,2.55,13588.0,United Kingdom
944466,575312,22037,ROBOT BIRTHDAY CARD,12,2011-11-09,0.42,13588.0,United Kingdom


## Интересующее событие - количество купленных товаров в чеках каждого покупателя в первые три дня после новогодних праздников,
## События наблюдения - страна, где сделана покупка; средняя цена товара

In [5]:
df=raw_data[raw_data.InvoiceDate.str.contains(r'20\d\d-01-0[4-6]', regex= True, na=False)]  # Выбираем даты только с 4 по 6 янаря
#Выбраны именно даты с 4 по 6 января, так как с 1 по 3 января данных нет

In [6]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
44722,493410,TEST001,This is a test product.,5,2010-01-04,4.50,12346.0,United Kingdom
44723,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04,4.25,14590.0,United Kingdom
44724,493412,TEST001,This is a test product.,5,2010-01-04,4.50,12346.0,United Kingdom
44725,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04,0.85,NaN,United Kingdom
44726,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04,3.75,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
543115,540397,22178,VICTORIAN GLASS HANGING T-LIGHT,6,2011-01-06,1.25,13187.0,United Kingdom
543116,540397,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,1,2011-01-06,2.95,13187.0,United Kingdom
543117,540397,84406B,CREAM CUPID HEARTS COAT HANGER,1,2011-01-06,3.25,13187.0,United Kingdom
543118,540397,22794,SWEETHEART WIRE MAGAZINE RACK,1,2011-01-06,7.95,13187.0,United Kingdom


In [7]:
#Группируем по ID покупателя, суммирая количество товаров в чеке и подсчитывая среднюю цену товара в чеке
df_group=df.groupby(['Customer ID', 'Country']).agg({'Price': ['mean'], 'Quantity': ['sum']}).reset_index()
df_group.columns = df_group.columns.droplevel(1)

In [8]:
df_group

,Customer ID,Country,Price,Quantity
0,12346.0,United Kingdom,4.500000,10
1,12386.0,Australia,1.050000,140
2,12395.0,Belgium,2.950000,-1
3,12413.0,France,5.640000,177
4,12415.0,Australia,1.871304,4662
...,...,...,...,...
201,18109.0,United Kingdom,4.706190,51
202,18145.0,United Kingdom,0.715333,676
203,18171.0,United Kingdom,5.011111,96
204,18226.0,United Kingdom,3.071905,366


In [9]:
df_group['Quantity']=(df_group['Quantity']//100+1)*100  # Разбиваем количество товаров в чеке на интервалы с шагом 100
df_group['Price']=(df_group['Price']//1+1)
df_group

,Customer ID,Country,Price,Quantity
0,12346.0,United Kingdom,5.0,100
1,12386.0,Australia,2.0,200
2,12395.0,Belgium,3.0,0
3,12413.0,France,6.0,200
4,12415.0,Australia,2.0,4700
...,...,...,...,...
201,18109.0,United Kingdom,5.0,100
202,18145.0,United Kingdom,1.0,700
203,18171.0,United Kingdom,6.0,100
204,18226.0,United Kingdom,4.0,400


In [10]:
# Создаём датафреймы отдельно для уникальных стран и для интервалов количества товаров в чеках
countries=df_group['Country'].unique()
prices=df_group['Price'].unique()
quantities=df_group['Quantity'].unique()
prices.sort()
quantities.sort()

df_countries=pd.DataFrame(data=countries, columns=['Country']) # Для стран
df_countries['key']=0
df_prices=pd.DataFrame(data=prices, columns=['Price']) # Для цен
df_prices['key']=0
df_quantities=pd.DataFrame(data=quantities, columns=['Quantity']) # Для количества
df_quantities['key']=0

In [11]:
# Создаём с помощью cross join таблицу совместного распределения 3 параметров
prob_temp=df_countries.merge(df_prices, on='key', how='outer') # Сначала cross join для стран и цен
prob=prob_temp.merge(df_quantities, on='key', how='outer') # Затем cross join с количеством
prob['Probability']=0
prob

,Country,key,Price,Quantity,Probability
0,United Kingdom,0,1.0,-200,0
1,United Kingdom,0,1.0,-100,0
2,United Kingdom,0,1.0,0,0
3,United Kingdom,0,1.0,100,0
4,United Kingdom,0,1.0,200,0
...,...,...,...,...,...
3427,EIRE,0,1127.0,2600,0
3428,EIRE,0,1127.0,2700,0
3429,EIRE,0,1127.0,3800,0
3430,EIRE,0,1127.0,4700,0


In [12]:
df_len=len(df_group)  # Общее число опытов

In [13]:
# Функция, подсчитывающая количество строк для определённой страны и количества товаров в чеке
def count_row(df_group, country, price, quantity):
    df_temp=df_group[(df_group['Country']==country)&(df_group['Price']==price)&(df_group['Quantity']==quantity)]
    return len(df_temp)

In [14]:
# Заполняем таблицу вероятностей
for i, row in prob.iterrows():
    temp_country=row['Country']
    temp_price=row['Price']
    temp_quantity=row['Quantity']
    result=count_row(df_group, temp_country, temp_price, temp_quantity)
    prob.iat[i, 4] = result/df_len

## Результат расчётов полного совместного распределения

In [15]:
prob.drop(['key'], axis=1, inplace = True)
prob = prob.sort_values(by=['Country', 'Price', 'Quantity'], ascending=[False, True, True])
prob

,Country,Price,Quantity,Probability
0,United Kingdom,1.0,-200,0.000000
1,United Kingdom,1.0,-100,0.000000
2,United Kingdom,1.0,0,0.009709
3,United Kingdom,1.0,100,0.004854
4,United Kingdom,1.0,200,0.009709
...,...,...,...,...
619,Australia,1127.0,2600,0.000000
620,Australia,1127.0,2700,0.000000
621,Australia,1127.0,3800,0.000000
622,Australia,1127.0,4700,0.000000


## Подсчёт маргинального распределения для стран

In [16]:
marg_dist_country = prob.groupby(by=['Country']).agg({'Probability': ['sum']}).reset_index()  # Группировка по странам с суммированием вероятностей
marg_dist_country.columns = marg_dist_country.columns.droplevel(1)
marg_dist_country.sort_values(by=['Probability'], ascending=False)

,Country,Probability
10,United Kingdom,0.902913
3,France,0.029126
4,Germany,0.014563
0,Australia,0.009709
1,Belgium,0.009709
2,EIRE,0.009709
5,Italy,0.004854
6,Japan,0.004854
7,Portugal,0.004854
8,Spain,0.004854


## Подсчёт маргинального распределения для цен

In [17]:
marg_dist_price = prob.groupby(by=['Price']).agg({'Probability': ['sum']}).reset_index()  # Группировка по странам с суммированием вероятностей
marg_dist_price.columns = marg_dist_price.columns.droplevel(1)
marg_dist_price.sort_values(by=['Probability'], ascending=False)

,Price,Probability
2,3.0,0.237864
3,4.0,0.208738
1,2.0,0.189320
4,5.0,0.145631
5,6.0,0.067961
0,1.0,0.048544
7,8.0,0.038835
6,7.0,0.024272
10,11.0,0.014563
8,9.0,0.009709


## Подсчёт маргинального распределения для количества товаров в чеке

In [18]:
marg_dist_quantity = prob.groupby(by=['Quantity']).agg({'Probability': ['sum']}).reset_index() # Группировка по количеству с суммированием вероятностей
marg_dist_quantity.columns = marg_dist_quantity.columns.droplevel(1)
marg_dist_quantity.sort_values(by=['Probability'], ascending=False)

,Quantity,Probability
3,100,0.199029
4,200,0.189320
2,0,0.155340
5,300,0.126214
6,400,0.077670
7,500,0.053398
8,600,0.053398
9,700,0.019417
10,800,0.019417
14,1200,0.014563


## Подсчёт маргинального распределения для стран и цен

In [19]:
marg_dist_country_price = prob.groupby(by=['Country','Price']).agg({'Probability': ['sum']}).reset_index()  # Группировка по странам с суммированием вероятностей
marg_dist_country_price.columns = marg_dist_country_price.columns.droplevel(1)
marg_dist_country_price.sort_values(by=['Probability'], ascending=False)

,Country,Price,Probability
132,United Kingdom,3.0,0.228155
133,United Kingdom,4.0,0.194175
131,United Kingdom,2.0,0.174757
134,United Kingdom,5.0,0.131068
135,United Kingdom,6.0,0.053398
...,...,...,...
45,France,7.0,0.000000
41,France,3.0,0.000000
40,France,2.0,0.000000
39,France,1.0,0.000000


## Подсчёт условного распределения для средней цены товара (условие - страна)

In [20]:
# Делаем удобную шапку таблицы (каждая колонка со страной соответствует условным вероятностям по этой стране)
header_countries = pd.MultiIndex(levels=[['Price','Probability'], np.concatenate(([''], countries))],
                      codes = [np.concatenate(([0], np.ones(len(countries)))),
                               np.arange(0, len(countries)+1, 1)]) 

In [21]:
cond_dist_price = pd.DataFrame(columns = header_countries)
cond_dist_price['Price'] = prices
for i, row in cond_dist_price.iterrows():
    for country in countries:
        joint_prob = float(marg_dist_country_price[(marg_dist_country_price['Country']==country)&(marg_dist_country_price['Price']==float(row['Price']))]['Probability']) # Совместная вероятность
        marg_prob = float(marg_dist_country[marg_dist_country['Country']==country]['Probability']) # Маргинальная вероятность
        cond_dist_price.iat[i, cond_dist_price.columns.get_loc(('Probability', country))] = (joint_prob/marg_prob)

In [22]:
cond_dist_price

Price    Probability                                                     \
           United Kingdom Australia Belgium    France   Germany Sweden Spain   
0      1.0       0.053763       0.0     0.0       0.0       0.0    0.0   0.0   
1      2.0       0.193548       1.0     0.0       0.0       0.0    0.0   0.0   
2      3.0       0.252688       0.0     0.5       0.0       0.0    0.0   0.0   
3      4.0       0.215054       0.0     0.0  0.333333       0.0    0.0   0.0   
4      5.0       0.145161       0.0     0.0  0.166667  0.666667    0.0   0.0   
5      6.0        0.05914       0.0     0.0  0.166667  0.333333    0.0   0.0   
6      7.0       0.016129       0.0     0.5       0.0       0.0    1.0   0.0   
7      8.0       0.037634       0.0     0.0       0.0       0.0    0.0   0.0   
8      9.0       0.010753       0.0     0.0       0.0       0.0    0.0   0.0   
9     10.0       0.005376       0.0     0.0       0.0       0.0    0.0   0.0   
10    11.0       0.005376       0.0     0.0  0.333333       0.0    0.0   0.0   
11    14.0       0.005376       0.0     0.0       0.0       0.0    0.0   0.0   
12  1127.0            0.0       0.0     0.0       0.0       0.0    0.0   1.0   

                              
   Italy Japan Portugal EIRE  
0    0.0   0.0      0.0  0.0  
1    0.0   1.0      0.0  0.0  
2    1.0   0.0      0.0  0.0  
3    0.0   0.0      1.0  0.0  
4    0.0   0.0      0.0  0.0  
5    0.0   0.0      0.0  0.5  
6    0.0   0.0      0.0  0.0  
7    0.0   0.0      0.0  0.5  
8    0.0   0.0      0.0  0.0  
9    0.0   0.0      0.0  0.0  
10   0.0   0.0      0.0  0.0  
11   0.0   0.0      0.0  0.0  
12   0.0   0.0      0.0  0.0

## Подсчёт условного распределения для страны (условие - средняя цена товара)

In [23]:
# Делаем удобную шапку таблицы (каждая колонка со страной соответствует условным вероятностям по этой стране)
header_prices = pd.MultiIndex(levels=[['Country','Probability'], np.concatenate(([''], prices))],
                      codes = [np.concatenate(([0], np.ones(len(prices)))),
                               np.arange(0, len(prices)+1, 1)]) 

In [24]:
cond_dist_country = pd.DataFrame(columns = header_prices)
cond_dist_country['Country'] = countries
for i, row in cond_dist_country.iterrows():
    for price in prices:
        joint_prob = float(marg_dist_country_price[(marg_dist_country_price['Price']==price)&(marg_dist_country_price['Country']==row['Country'][0])]['Probability']) # Совместная вероятность
        marg_prob = float(marg_dist_price[marg_dist_price['Price']==price]['Probability']) # Маргинальная вероятность
        cond_dist_country.iat[i, cond_dist_country.columns.get_loc(('Probability', str(price)))] = (joint_prob/marg_prob)

In [25]:
cond_dist_country

Country Probability                                          \
                           1.0       2.0       3.0       4.0       5.0   
0   United Kingdom         1.0  0.923077  0.959184  0.930233       0.9   
1        Australia         0.0  0.051282       0.0       0.0       0.0   
2          Belgium         0.0       0.0  0.020408       0.0       0.0   
3           France         0.0       0.0       0.0  0.046512  0.033333   
4          Germany         0.0       0.0       0.0       0.0  0.066667   
5           Sweden         0.0       0.0       0.0       0.0       0.0   
6            Spain         0.0       0.0       0.0       0.0       0.0   
7            Italy         0.0       0.0  0.020408       0.0       0.0   
8            Japan         0.0  0.025641       0.0       0.0       0.0   
9         Portugal         0.0       0.0       0.0  0.023256       0.0   
10            EIRE         0.0       0.0       0.0       0.0       0.0   

                                                          
         6.0  7.0    8.0  9.0 10.0      11.0 14.0 1127.0  
0   0.785714  0.6  0.875  1.0  1.0  0.333333  1.0    0.0  
1        0.0  0.0    0.0  0.0  0.0       0.0  0.0    0.0  
2        0.0  0.2    0.0  0.0  0.0       0.0  0.0    0.0  
3   0.071429  0.0    0.0  0.0  0.0  0.666667  0.0    0.0  
4   0.071429  0.0    0.0  0.0  0.0       0.0  0.0    0.0  
5        0.0  0.2    0.0  0.0  0.0       0.0  0.0    0.0  
6        0.0  0.0    0.0  0.0  0.0       0.0  0.0    1.0  
7        0.0  0.0    0.0  0.0  0.0       0.0  0.0    0.0  
8        0.0  0.0    0.0  0.0  0.0       0.0  0.0    0.0  
9        0.0  0.0    0.0  0.0  0.0       0.0  0.0    0.0  
10  0.071429  0.0  0.125  0.0  0.0       0.0  0.0    0.0

## Подсчёт условного распределения для количества товара (условия - количество товаров в чеке и средняя цена товара)

## Подсчёт ведётся по цепному правилу P(x_1, x_2, x_3) = P(x_1)P(x_2|x_1)P(x_3|x_1,x_2)

In [26]:
cond_dist_quantity = prob

In [27]:
for i, row in cond_dist_quantity.iterrows():
    temp_country=row['Country']
    temp_price=row['Price']
    temp_prob=row['Probability']
    marg_prob=float(marg_dist_country[marg_dist_country['Country']==temp_country]['Probability'])
    cond_prob=float(cond_dist_price[cond_dist_price['Price']==temp_price]['Probability', temp_country])
    if temp_prob:
        prob.iat[i, 3]=float(temp_prob/(marg_prob*cond_prob))

In [28]:
cond_dist_quantity[cond_dist_quantity['Probability']!=0]

,Country,Price,Quantity,Probability
2,United Kingdom,1.0,0,0.200000
3,United Kingdom,1.0,100,0.100000
4,United Kingdom,1.0,200,0.200000
8,United Kingdom,1.0,600,0.200000
9,United Kingdom,1.0,700,0.100000
...,...,...,...,...
770,Belgium,7.0,0,0.004854
340,Australia,2.0,200,0.004854
358,Australia,2.0,4700,0.004854
433,Australia,6.0,-100,1.000000


In [65]:
data = cond_dist_quantity
data

,Country,Price,Quantity,Probability
0,United Kingdom,1.0,-200,0.0
1,United Kingdom,1.0,-100,0.0
2,United Kingdom,1.0,0,0.2
3,United Kingdom,1.0,100,0.1
4,United Kingdom,1.0,200,0.2
...,...,...,...,...
619,Australia,1127.0,2600,0.0
620,Australia,1127.0,2700,0.0
621,Australia,1127.0,3800,0.0
622,Australia,1127.0,4700,0.0


In [66]:
# Преобразование стран и даты заказа (строчный тип) в числовой признак
le = LabelEncoder()
data['Country'] = le.fit_transform(data['Country'])

# Разделение на тренировочную и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Выбор признаков для модели
#features = ['Customer ID','Country', 'InvoiceDate', 'Price']

# Обучение модели
y_test = test_data['Quantity']
y_pred = test_data['Probability']

# Прогнозирование на тестовой выборке
logloss = log_loss(y_test, y_pred)
print(f"Логарифмическая потеря (Log Loss): {logloss}")

ValueError: y_true and y_pred contain different number of classes 24, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [-200 -100    0  100  200  300  400  500  600  700  800  900 1000 1100
 1200 1600 1700 1800 2000 2600 2700 3800 4700 8200]

In [59]:
# Оценка ошибки прогноза
mse = mean_squared_error(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Среднеквадратичная ошибка (MSE): {mse}")
print(f"Средняя абсолютная ошибка (MAE): {mae}")
print(f"Коэффициент детерминации (R-squared): {r2}")
print(f"Средняя абсолютная процентная ошибка (MAPE): {mape}%")

Среднеквадратичная ошибка (MSE): 6317.704379809221
Средняя абсолютная ошибка (MAE): 31.43282988871225
Коэффициент детерминации (R-squared): 0.3043185954659613
Средняя абсолютная процентная ошибка (MAPE): inf%
